<a href="https://colab.research.google.com/github/sahil350/sc2x-edx/blob/main/network_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Network Models

In [7]:
!pip install -q ortools

     |████████████████████████████████| 14.0MB 302kB/s 
     |████████████████████████████████| 1.0MB 61.4MB/s 
     |████████████████████████████████| 133kB 54.2MB/s 
ERROR: tensorflow-metadata 0.26.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.


In [8]:
import pandas as pd
import numpy as np
import networkx as nx
from plotly import graph_objects as go
from ortools.linear_solver import pywraplp

## The Transportations Problem

### Steel Co 

Steel Co supplying to 4 construction projects in Northeast regions:


1.  Buffalo: 40,500 pounds
2.  Albany: 22,230 pounds
3.  Philadelphia: 85,200 pounds
4.  Cleveland: 47,500 pounds

Two distribution centers 
1. Allentown: 100,000 pounds
2. Toledo: 100,000 pounds

#### Cost of transportation from DC to Project Center

DC| **Buffalo** | **Albany** | **Philadelphia** | **Cleveland**
--- | --- | --- | --- | ---
Allentown | 52 | 32 | 11 | 69
Toledo | 45 | 84 | 76 | 15




In [9]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [10]:
# add variables 
# key
# x11 : Allentown to Buffalo
# x12 : Allentown to Albany
# x13 : Allentown to Philadelphia
# x14 : Allentown to Cleveland
# x21 : Toledo to Buffalo
# x22 : Toledo to Albany
# x23 : Toledo to Philadelphia
# x24 : Toledo to Cleveland
# All xij are in pounds of steel

dec_vars = {}
for var in ['x11', 'x12', 'x13', 'x14', 'x21', 'x22', 'x23', 'x24']:
  dec_vars[var] = solver.NumVar(0, solver.infinity(), var)

In [11]:
# add supply constraints
# There will be two constraints
# constraints : Sum(xij) <= 100000
# for all j in {1, 2, 3, 4}
# for all i in {1, 2}

for i in [1, 2]:
  solver.Add(dec_vars[f'x{i}1'] + dec_vars[f'x{i}2'] + dec_vars[f'x{i}3'] + \
             dec_vars[f'x{i}4'] <= 100000)


In [12]:
# add demand constraints
# There will be 4 constraints
# constraints : Sum(xij) >= dj
# for all j in {1, 2, 3, 4}
# for all i in {1, 2}
# where dj are corresponding demands

dj = {
    'd1': 40500,
    'd2': 22230,
    'd3': 85200,
    'd4': 47500
}

# constraints
for j in [1, 2, 3, 4]:
  solver.Add(dec_vars[f'x1{j}'] + dec_vars[f'x2{j}'] >= dj[f'd{j}']) 

In [14]:
# add objective function
# we want to minimize cost
# obj: Min Sum(cij*xij)
# for all j in {1, 2, 3, 4}
# for all i in {1, 2}

costs = {
    'c11' : 52,
    'c12' : 32,
    'c13' : 11,
    'c14' : 69,
    'c21' : 45,
    'c22' : 84,
    'c23' : 76,
    'c24' : 15
}

obj_fun = 0

for i in [1, 2]:
  for j in [1, 2, 3, 4]:
    obj_fun += costs[f'c{i}{j}'] * dec_vars[f'x{i}{j}']

solver.Minimize(obj_fun)

In [18]:
# solve LP
status = solver.Solve()

In [31]:
# display results
display_dict = {
    'x11' : 'Allentown to Buffalo',
    'x12' : 'Allentown to Albany',
    'x13' : 'Allentown to Philadelphia',
    'x14' : 'Allentown to Cleveland',
    'x21' : 'Toledo to Buffalo',
    'x22' : 'Toledo to Albany',
    'x23' : 'Toledo to Philadelphia',
    'x24' : 'Toledo to Cleveland'
}
if status == pywraplp.Solver.OPTIMAL:
  print(f'Minimum Cost: $ {solver.Objective().Value()}')
  for i in [1, 2]:
    for j in [1, 2, 3, 4]:
      print(f'{display_dict[f"x{i}{j}"]}:' +
      f' {round(dec_vars[f"x{i}{j}"].solution_value(), 2)} pounds')
  allentown = dec_vars['x11'].solution_value() +\
              dec_vars['x12'].solution_value() +\
              dec_vars['x13'].solution_value() +\
              dec_vars['x14'].solution_value()

  toledo = dec_vars['x21'].solution_value() +\
           dec_vars['x22'].solution_value() +\
           dec_vars['x23'].solution_value() +\
           dec_vars['x24'].solution_value()  
  print(f'Total shipped from Allentown: {round(allentown, 2)} pounds')
  print(f'Total shipped from Toled: {round(toledo, 2)} pounds')

Minimum Cost: $ 4569920.0
Allentown to Buffalo: 0.0 pounds
Allentown to Albany: 14800.0 pounds
Allentown to Philadelphia: 85200.0 pounds
Allentown to Cleveland: 0.0 pounds
Toledo to Buffalo: 40500.0 pounds
Toledo to Albany: 7430.0 pounds
Toledo to Philadelphia: 0.0 pounds
Toledo to Cleveland: 47500.0 pounds
Total shipped from Allentown: 100000.0 pounds
Total shipped from Toled: 95430.0 pounds
